# Eventful Lists

keeping track of element changes in a list

In [1]:
from spectate import watched_type

We use `watched_type` to create an `EventfulDict` that enables callbacks before and after an element of the dictionary is set. The funtion's signature requries the name of the new eventful type, a base class from which that new type should inherit, and the series of attributes on that base class which can register a callback. In this case we enable callbacks for methods which can set or delete items:

+ `__setitem__`
+ `__delitem__`
+ `pop`

In [2]:
EventfulList = watched_type('EventfulList', list, '__setitem__', '__delitem__', 'pop')

Beforebacks
-----------

+ Have a signature of ``(instance, call)``

    + ``instance`` is the owner of the method
    + ``call`` is a ``Bunch`` with the keys:

        + ``'name'`` - the name of the method which was called
        + ``'args'`` - the arguments which that method will call
        + ``'kwargs'`` - the keywords which that method will call

+ Can ``return`` a value which gets passed on to its respective afterback.
+ If an error is encountered, the wrapper will ``return`` the original ``call``
as if it were that of the beforeback, and set ``error`` in the ``answer``
bunch of its afterback. This way, if the base method call is valid, it's not
obstructed by a raised beforeback. Thus, beforeback errors should be handled
in an afterback.

Afterbacks
----------

+ Have a signature of ``(instance, answer)``

    + ``instance`` is the owner of the method
    + ``answer`` is a ``Bunch`` with the keys:

        + ``'name'`` - the name of the method which was called
        + ``'value'`` - the value returned by the method
        + ``'before'`` - the value returned by the respective beforeback
        + ``'error'`` - None, or the error encountered in the beforeback

+ Responcible for raising beforeback errors.
+ Should not ``return``

In [3]:
# beforeback

def pass_on_old_value(inst, call):
    index, old = call.args[0], inst[call.args[0]]
    return index, old

# afterback

def print_element_change(inst, answer):
    # answer.before = pass_on_old(call)
    index, old = answer.before
    try:
        new = inst[index]
    except:
        new = ''
    if new != old:
        print("{%s: %s} -> {%s: %s}" %
            (index, old, index, new))

Instances of `EventfulList` have a public method `spectator_callback` which is used to store unique callback pairs. 

Each pair can have a beforeback, and/or an afterback which are specified in the method signature with the keywords `before` and `after`.

In [4]:
elist = EventfulList([1, 2, 3])

elist.spectator_callback('pop',
    '__delitem__', '__setitem__',
    before=pass_on_old_value,
    after=print_element_change)

In [5]:
# does not notify because
# values are the same
elist[0] = 1

In [6]:
# will notify when the
# values are different
elist[0] = 0

{0: 1} -> {0: 0}


In [7]:
# notify when deleting
# an element of the list
elist.pop(0)
del elist[0]

{0: 0} -> {0: 2}
{0: 2} -> {0: 3}


In [8]:
elist

[3]